# Process Data for D3 Visualizations

In [2]:
import os
import pandas as pd
import json
# import numpy as np

## Merged Data

In [3]:
data_path = 'D:\DATA\OurFoods'
df = pd.read_csv(os.path.join(data_path, 'merged_amz-off_3.csv.gz'),\
                 dtype={'customer_id': 'object', 'product_parent': 'object', \
                        'star_rating': pd.Int64Dtype(), 'helpful_votes': pd.Int64Dtype(), 
                        'total_votes': pd.Int64Dtype(), \
                        'code': 'object'},
                 compression='gzip')
# convert reivew_date to datetime object
df.review_date = pd.to_datetime(df.review_date)
df.shape

(147304, 23)

## Data to PostgreSQL

In [6]:
import os
from sqlalchemy import create_engine
from sqlalchemy import types

from dotenv import load_dotenv # env variables
load_dotenv(verbose=True)

True

In [7]:
SQLALCHEMY_DATABASE_URI = os.getenv('DATABASE_URL')
engine = create_engine(SQLALCHEMY_DATABASE_URI)

> Column Type inherited from DataFrame `info`
+ object => text
+ Int64 => bigint
+ datetime64[ns] => timestamp without time zone 
+ float64 => double precision   

In [19]:
df.to_sql(name="sample_table", con=engine, if_exists='replace',\
          schema='public', index=False)

In [20]:
# add review_id as primary key
engine.execute("ALTER TABLE sample_table ADD PRIMARY KEY (review_id);")

+ create a connection, query then close it:
```python
connection = engine.connect()  
result = connection.execute("select * from skilltree")
for row in result:
    print(row)
connection.close()
```

+ engine create and close the connectio itself:
```python
result = engine.execute("select * from skilltree")
for row in result:
    print(row)
```

## Data for Ridgeline Plot
+ Input: 
  + Merged Food Reviews dataset
  + A range of time, including "start date" and "end date"
+ Ouput:
  + Data of TOP10 Counts within the time range
  + Including these attributes:
    + For each category, data of one day is computed
      + **p: probability of reviews in a given date**
        + i.e. (count of day reviews) / (count of whole time reivews)
      + **p peak: normalize p with the maximum p (of whole range)**
        + i.e. (p of a date) / (max p of whole time range)
      + **p smooth: smooth p with adjacent p data**
        + i.e. (p of a date) / (sum of p data adjacent to it, including previous, current, next)
      

In [25]:
df.shape

(147304, 23)

### Preparation of Data
+ Keep only Valid Data
+ Slice by Date Range
+ Slice by TOP 10 Threshold

+ Pattern Matching, w/ SIMILAR TO
  + https://www.postgresql.org/docs/9.0/functions-matching.html
+ give `%%` to use it as `%` because `%` in python is use as string formatting 

In [292]:
start_date = '2014-01-01'
end_date = '2014-12-31'

In [293]:
query = \
"""
SELECT 
    main_category_en AS category, 
    review_date, 
    review_id AS id
FROM 
    sample_table
WHERE 
    energy_100g IS NOT NULL
    AND review_date IS NOT NULL
    AND main_category_en IS NOT NULL
    AND energy_100g < 3000
    AND salt_100g < 100
    AND main_category_en SIMILAR TO '[A-Z]_*'
    AND review_date BETWEEN '{0}' AND '{1}'
ORDER BY
    review_date
""".format(start_date, end_date)

In [ ]:
sub = pd.read_sql(query, con=engine)

In [294]:
# get top 10 threshold
threshold = sub.groupby('category')[['id']].count()\
    .sort_values('id', ascending=False)\
    .iloc[9, 0] # get 10th category count
threshold

233

In [285]:
# Sice by TOP10 threshold
top10 = sub.assign(counts=lambda d: d.groupby('category')[['id']].transform('count'))\
        .query('counts >= {}'.format(threshold))\
        .reset_index(drop=True)
top10.shape

(16048, 4)

### Date Range index
+ Not Every Category has data for every date in the range
+ Supplement with 0 value 
+ Use Date Index

In [286]:
date_idx = []
for category in top10.category.unique():
    for date in pd.date_range(start_date, end_date, freq='D'):
        date_idx.append((category, date))
len(date_idx) # (number of category) * (number of days)

3650

### Calculate Plot Values
+ P
+ P Peak
+ P Smooth (use more days for smoothing?)

In [287]:
# add p attribute
data = top10.groupby(['category', 'review_date'])[['id']].count()\
    .reindex(date_idx, fill_value=0)\
    .reset_index()\
    .assign(byCategorySum=lambda d: d.groupby('category')[['id']].transform('sum'))\
    .assign(p=lambda d: d.id / d.byCategorySum)\
    .drop(['id', 'byCategorySum'], axis=1)
data.shape

(3650, 3)

In [288]:
# add p peak attribute
data = data.assign(byCategoryMaxP=lambda d: d.groupby('category')[['p']].transform(max))\
    .assign(p_peak=lambda d: d.p / d.byCategoryMaxP)\
    .drop(['byCategoryMaxP'], axis=1)
data.shape

(3650, 4)

In [289]:
# add p smooth attribute
data = data.assign(p_lag1=lambda d: d.groupby('category')[['p_peak']].shift(-1))\
    .assign(p_lead1=lambda d: d.groupby('category')[['p_peak']].shift(1))\
    .assign(p_smooth=lambda d: (d.p_lag1 + d.p_peak + d.p_lead1) / 3)\
    .drop(['p_lag1', 'p_lead1'], axis=1)\
    .fillna(method='ffill', axis=1)
data.shape

(3650, 5)

In [290]:
# add p smooth attribute, w/ 7 days smoothing
data = data\
    .assign(p_lag1=lambda d: d.groupby('category')[['p_peak']].shift(-1))\
    .assign(p_lag2=lambda d: d.groupby('category')[['p_peak']].shift(-2))\
    .assign(p_lag3=lambda d: d.groupby('category')[['p_peak']].shift(-3))\
    .assign(p_lead1=lambda d: d.groupby('category')[['p_peak']].shift(1))\
    .assign(p_lead2=lambda d: d.groupby('category')[['p_peak']].shift(2))\
    .assign(p_lead3=lambda d: d.groupby('category')[['p_peak']].shift(3))\
    .assign(p_smooth7=lambda d: (d.p_lag1 + d.p_lag2 + d.p_lag3 + 
                                 d.p_lead1 + d.p_lead2 + d.p_lead3 +
                                 d.p_peak) / 7)\
    .drop(['p_lag1', 'p_lag2', 'p_lag3', 'p_lead1', 'p_lead2', 'p_lead3'], axis=1)\
    .fillna(method='ffill', axis=1)
data.shape

(3650, 6)

## Data for Statcked Area Plot
+ Input: 
  + Merged Food Reviews dataset
  + A range of time, including "start date" and "end date"
+ Ouput:
  + Data of TOP10 Counts w/ star rating within the time range
  + Including these attributes:


In [292]:
start_date = '2014-01-01'
end_date = '2014-12-31'

In [310]:
query = \
"""
SELECT 
    main_category_en AS category, 
    review_date, 
    review_id AS id,
    star_rating
FROM 
    sample_table
WHERE 
    energy_100g IS NOT NULL
    AND review_date IS NOT NULL
    AND main_category_en IS NOT NULL
    AND energy_100g < 3000
    AND salt_100g < 100
    AND main_category_en SIMILAR TO '[A-Z]_*'
    AND review_date BETWEEN '{0}' AND '{1}'
ORDER BY
    review_date
""".format(start_date, end_date)

In [311]:
sub = pd.read_sql(query, con=engine)
sub.shape

(17513, 4)

In [312]:
# get top 10 threshold
threshold = sub.groupby('category')[['id']].count()\
    .sort_values('id', ascending=False)\
    .iloc[9, 0] # get 10th category count
threshold

233

In [313]:
from pandas.tseries.offsets import MonthEnd

In [314]:
# Sice by TOP10 threshold and add attribute of each month (date)
top10 = sub.assign(counts=lambda d: d.groupby('category')[['id']].transform('count'))\
        .query('counts >= {}'.format(threshold))\
        .assign(year=lambda d: d.review_date.dt.year, month=lambda d: d.review_date.dt.month)\
        .assign(date=lambda d: pd.to_datetime({'year': d.year, 'month': d.month, 'day': 1}) + MonthEnd(0))\
        .drop(['review_date', 'year', 'month'], axis=1)\
        .reset_index(drop=True)
top10.shape

(16048, 5)

In [315]:
top10.head()

,category,id,star_rating,counts,date
0,Plant-based foods and beverages,R2R89SB5NTN2S4,5,4330,2014-01-31
1,Plant-based foods and beverages,R3ITMBCU6TYFNK,1,4330,2014-01-31
2,Beverages,RJ73WTW1DYKB6,5,4313,2014-01-31
3,Snacks,R1XZA5DARTY6UC,1,3487,2014-01-31
4,Snacks,RN1JCBRZTJQBH,1,3487,2014-01-31


### Date Range index
+ Not Every Product of a Category has data for the given range
+ Use date index to fill the gaps, and fill with 0

In [19]:
date_idx = []
for category in top10.category.unique():  # category
    for rating in range(1, 6): # rating from 1 to 5
        for date in pd.date_range(start_date, end_date, freq='M'):  # months in given time
            date_idx.append((category, rating, date))
len(date_idx) # (number of category) * (number of product in a category) * (number of days)

600

In [31]:
# date of rating count, of a category, by each month
data = top10.groupby(['category', 'star_rating', 'date'])[['id']].count()\
    .reindex(date_idx, fill_value=0)\
    .unstack(level=[0, 1])  # unstack to move indices to columns
# drop the id column
data.columns = data.columns.droplevel(level=0)
# remove index name
# data.index.name = None
data

category    Plant-based foods and beverages                  Beverages      \
star_rating                               1   2   3   4    5         1   2   
date                                                                         
2014-01-31                               18  13  19  40  247        23   9   
2014-02-28                               15  10  17  29  215        14   9   
2014-03-31                               18  10  25  55  268        15  16   
2014-04-30                               24  10  13  35  181        23  11   
2014-05-31                               10   7  14  32  183        23   7   
2014-06-30                               18   4  15  28  180        24  16   
2014-07-31                               26  16  33  48  296        20  11   
2014-08-31                               21   9  18  45  313        25  12   
2014-09-30                               21   7  17  53  313        19  15   
2014-10-31                               21  21  26  39  289        30  15   
2014-11-30                               24  11  29  47  290        32  27   
2014-12-31                               34  13  23  45  392        25  19   

category                  ... Dairies              Sweeteners               
star_rating   3   4    5  ...       1  2  3  4   5          1  2  3  4   5  
date                      ...                                               
2014-01-31   16  25  176  ...       2  1  0  5  14          1  1  3  2  13  
2014-02-28   13  24  183  ...       2  0  1  3  11          1  5  6  7  20  
2014-03-31   23  35  158  ...       0  1  1  2  16          3  2  3  6  33  
2014-04-30   21  28  210  ...       2  1  0  0   7          1  1  0  3  16  
2014-05-31   22  34  173  ...       0  0  2  2  14          0  2  6  2   9  
2014-06-30   19  48  190  ...       3  0  2  1  11          1  0  3  2  14  
2014-07-31   26  59  301  ...       1  1  2  2  17          1  1  0  2  18  
2014-08-31   24  70  348  ...       4  0  1  0   9          0  1  1  2  28  
2014-09-30   23  54  290  ...       0  0  1  3  13          5  3  1  3   9  
2014-10-31   25  48  292  ...       0  1  2  3  22          0  1  3  2  16  
2014-11-30   43  53  319  ...       1  0  1  5  16          4  1  2  4  24  
2014-12-31   19  69  391  ...       1  0  1  3  17          0  1  2  8  21  

[12 rows x 50 columns]

In [52]:
path = "D:\Dropbox\!FullStack-DataScientist\1_FRONT-END\D3\D3-FlaskServer\static\data\local.json"

In [54]:
data.to_json(path, orient='split')
# split into {'columns': ..., 'data': ..., 'index': ...}

OSError: [Errno 22] Invalid argument: 'D:\\Dropbox\\!FullStack-DataScientist\x01_FRONT-END\\D3\\D3-FlaskServer\\static\\data\\local.json'